# Assignment 3: object detection

In this assignment, we will localize chicory in an image and obtain the optimal cutting line of the chicory. We will use the YOLOv8 keypoint detector to do this. The dataset of images containing chicory contains labels with a bounding box and two points representing the cutting line. First, we will import all necessary modules that we need to get the job done. The next parts of the code are organized in: data analysis, data preprocessing, model training, and model validation. 

In [ ]:
# Imports
import os
import requests
from PIL import Image
from io import BytesIO
! pip install ultralytics > /dev/null
import ultralytics
from ultralytics import YOLO
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
%load_ext tensorboard

## Data analyis

In the first step, we will download the data. The data folder contains a folder with images and a folder with labels in the YOLO format. This format has a .txt file for every image with one line for each annotation. Each line contains the image class, the normalized bounding box coordinates [x_center, y_center, width, height], and the normalized coordinates of the two points [x1, y1, x2, y2]. We also use a .json file with all labels in COCO format for visualization of the labels.  

In [ ]:
# Download data
if not os.path.exists('data/chicory_data'):
    !wget -q --no-check-certificate -O dataset.zip "https://kuleuven-my.sharepoint.com/:u:/g/personal/matthias_deryck_kuleuven_be/EUD1b3ZjnixMnG9XLTAhBSABPVeLHHEuLhq6zbygg0nb_g?e=n5lOdk&download=1"
    !unzip -q dataset.zip -d data
    !rm dataset.zip

In [ ]:
# Create a COCO object from the COCO annotations file
coco = COCO('./data/chicory_data/witloof_dataset_coco.json')

# Get the annotations for the first image
annotations = coco.loadAnns(coco.getAnnIds(imgIds=1))

# Get the image info
img_info = coco.loadImgs(1)[0]
height, width = img_info['height'], img_info['width']
file_name = img_info['file_name']

# Load the image
response = requests.get(file_name)
image = Image.open(BytesIO(response.content))

# Pot annotations
plt.imshow(image); plt.axis('off')
coco.showAnns(annotations, draw_bbox=True)
plt.show()

## Data preprocessing

For a proper pipeline, we will split our total dataset into a training, validation, and testset. De model gets trained on images from the training set. After each batch, the model is validated on the validation set. After training, we can evaluate the final model on the testset. The below command creates three autosplit .txt files that contain the paths to the images of the corresponding dataset for training, validation, and testing. 

In [ ]:
# Split data in 80% training, 10% validation, and 10% test sets
ultralytics.data.utils.autosplit(path='./data/chicory_data/images', weights=(0.8, 0.1, 0.1), annotated_only=False)

## Model training

In the next step, we will train our model. The model that we use is a pretrained YOLOv8n-pose model that was trained on the COCO dataset. We use the Ultralytics API for the training and validation phase. More info on: https://docs.ultralytics.com/tasks/pose/. All model parameters are logged during training, all parameters can be visualized using Tensorboard which will be activated below. 

In [ ]:
# Run tensorboard
%tensorboard --logdir runs/pose/train

In [ ]:
# Load a YOLOv8 nano model for pose estimation
model = YOLO('yolov8n-pose.yaml').load('yolov8n-pose.pt')

# Training params
epochs = 20 # All data is sent 'epoch' times through the network
image_size = 640 # All images are resized to this size before entering the network

# Train the model on the data
results = model.train(data='./data/chicory_data/witloof_dataset.yaml', epochs=epochs, imgsz=image_size)

## Model validation

After training, we load the best model and perform a validation step on the test data. Afterwards, we show some predictions. 

In [ ]:
# Load the trained model
model = YOLO('./runs/pose/train/weights/best.pt')

In [ ]:
# Validate the model
metrics = model.val(split='test')

In [ ]:
# Predict with the model
results = model('./data/chicory_data/autosplit_test.txt', stream=True)  # predict on an image

# Plot results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    plt.imshow(im)
    plt.show()